In [ ]:
import socket
from concurrent import futures

def check_port(target_ip, port_number, timeout):
    TCP_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    TCP_sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    TCP_sock.settimeout(timeout)
    try:
        TCP_sock.connect((target_ip, port_number))
        return port_number
    except socket.timeout:
        return None
    except ConnectionRefusedError:
        return None
    finally:
        TCP_sock.close()

def port_scanner(target_ip, timeout):
    max_workers = 100  # Adjust this according to your system's capabilities
    ports_to_check = range(1, 65536)  # Scan all valid port numbers
    with futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        checks = [executor.submit(check_port, target_ip, port, timeout) for port in ports_to_check]
        for future in futures.as_completed(checks):
            port = future.result()
            if port:
                print('Listening on port: {}'.format(port))

def main():
    target_ip = input("Enter the target IP address: ")
    timeout = int(input("Enter the connection timeout (in seconds): "))
    port_scanner(target_ip, timeout)

if __name__ == "__main__":
    main()
